Extracción

In [6]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread 
import pandas as pd
import numpy as np


In [7]:
#Documentación
ruta_permisos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"          
alcances = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive'] 

#credenciales
cred = service_account.Credentials.from_service_account_file(ruta_permisos, scopes = alcances)

#Objeto de conexión
servicesheets = build('sheets','v4',credentials=cred)
servicedrive = build('drive','v3', credentials= cred)

gs = servicesheets.spreadsheets()

Tomaremos 3 cotizaciones y extraeremos su información para luego transformarla


In [ ]:
#Extracción de una cotización
#cotn[nombre,id]
cot1 = ["CTQ-1020 EATON BUSSMANN (SERVICIO DE RENTA DE PLATAFORMA TIJERA JLG MODELO 2030ES)","16h05uAcLjE7rqvFzD9Ij3PDMZcdg-bai5al5Ijz9r2o"]

nom_hoja = 'CARATULA'
ran = '!A1:H26'

extract1 = gs.values().get(
                spreadsheetId = cot1[1],
                range = nom_hoja+ran).execute()
        
       

#encabezados = extract1['values'][0]
data = extract1['values'][0:]
extraccion1 = pd.DataFrame(data)

print(extraccion1)

In [120]:
#extracción de 3 cotizaciones al mismo tiempo, con su respectivo nombre:
metadata = {'CTQ-1020 EATON BUSSMANN (SERVICIO DE RENTA DE PLATAFORMA TIJERA JLG MODELO 2030ES)' : '16h05uAcLjE7rqvFzD9Ij3PDMZcdg-bai5al5Ijz9r2o',
            'CTQ-1255 ZF (TRAMITOLOGIA Y GESTION DE INCREMENTOS DE CARGA Y DOCUMENTACION DE OBRAS ANTE CFE) (1)' : '15xpNv-lkcE5ufSMEtweBBAjUA0zv7Ghtv3f-xOlXLPE',
            'CTQ-1261 POLIGOF (DICTAMEN ELECTRICO)' : '1gY6FAEvBap8P7LqEPkemuFB2fBpXlpAhQEBtv_NOtvc'}

#definimos la estrctura que contendra los datos de la factura
#rango = 'A1:H26'

def extraccion(service, sp_id,range):
    res = gs.values().get(spreadsheetId = sp_id,range = nom_hoja + ran).execute()
    values = res.get('values', [])                  #convertimos los dataframes desde la función
    return pd.DataFrame(values) 
    

In [ ]:
#Creamos un diccionario para guardar los dataframes.
cotizaciones = {}
# for ids,nom_sheet in metadata.items():
#     print(f"este es el nombre: {ids} y este es su id: {nom_sheet}")

#datos = extraccion(gs,metadata['CTQ-1020 EATON BUSSMANN (SERVICIO DE RENTA DE PLATAFORMA TIJERA JLG MODELO 2030ES)'],ran)
#print(datos)



for nombre,id in metadata.items():
    cotizaciones[nombre]= extraccion(gs,id,ran)
    

print(cotizaciones)


Exportando:

Convertiremos los dataframes en CSV y los guardaremos en dos ubicaciones:

-On premise: dentro del equipo de computo.

-On cloud: dentro de Google Drive.

In [136]:
#ON PREMISE
#exportar dataframes a CSV          #Utilizaremos el modulo .to_csv()
import os   #libreria para interactuar con el sistema operativo (trabajar con archivos y directorios, rutas, etc.)

#Esta extracción guardara el CSV en la carpeta donde tienes el codigo almacenado  , Utilizaremos una ruta para almecenarlo ena carpeta
extraccion1.to_csv('extraccióncsv', sep=',' ,index=False, encoding='utf-8')


file_path ='C:/Users/52473/OneDrive/Documentos/SITEC/Cotizaciones/Capa1 extracciónCSV'
name = 'extraccion1csv.csv'
ruta_total = os.path.join(file_path, name)
extraccion1.to_csv(ruta_total, sep=',' ,index=False, encoding='utf-8')

In [ ]:
#ON CLOUD
#Tomaremos el objeto de conexión que creamos anteriormente (servicedrive).
folder_id = '1tS-6rXHVQna9CD3jafLnQ62Ps4CtZ9xO'

#guardaremos el dataframe en un archivo temporal en la extension /tmp

ruta_archivo = os.path.join('/tmp', name)
extraccion1.to_csv(ruta_archivo, index = False, encoding = 'UTF-8')

#Creamos un archivo vacio tipo .csv en el cual guardaremos el dataframe extraido



Extraemos las cotizaciones por lotes: 

los lotes seran 
2024:

Enero

Febrero

Marzo

Abril

Mayo